In [1]:
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
from torch.optim.lr_scheduler import CosineAnnealingLR

import torchio as tio
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torchio")

from utils.utils import train_3d, validate_3d
from utils.vis import plot_mri
from utils.dataset import BrainMRIDataset
from utils.loss import DiceCrossEntropyLoss, DiceFocalLoss
from models import UNet3D, AttentionUNet
from monai.networks.nets import UNet, SegResNet, UNETR

## Constants

In [2]:
ROOT_DIR = './Data'
BATCH_SIZE = 1
EPOCHS = 300
NUM_CLASSES = 4
NUM_WORKERS=16
DEVICE = 'mps' if torch.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'

## Transforms

In [3]:
# TorchIO transformations for augmentation
train_transform = tio.Compose([
    tio.RandomAffine(scales=(0.9, 1.1)),
    # tio.RandomElasticDeformation(num_control_points=(7, 7, 7), max_displacement=(4, 4, 4)),
    # tio.RandomFlip(axes=(0, 1, 2)),
    # tio.RandomBiasField(coefficients=(0.1, 0.3)),
    tio.RescaleIntensity((0, 1))  # Normalize intensity to [0, 1]
])

val_transform = tio.Compose([
    tio.RescaleIntensity((0, 1))  # Only normalize intensity for validation
])

In [4]:
# Create datasets
train_dataset = BrainMRIDataset(os.path.join(ROOT_DIR, 'train'), transform=train_transform)
val_dataset = BrainMRIDataset(os.path.join(ROOT_DIR, 'val'), transform=val_transform)

# Create DataLoaders
train_loader = tio.SubjectsLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = tio.SubjectsLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

## Visualization

In [ ]:
slice_idx = 100
for batch in val_loader:
    images, masks = batch["image"]["data"], batch["mask"]["data"]
    print(images.shape, masks.shape)
    
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(images[0, 0, :, :, slice_idx].numpy(), cmap='gray')
    axs[0].set_title('image')
    axs[1].imshow(masks[0, 0, :, :, slice_idx].numpy(), cmap='gray')
    axs[1].set_title('mask')
    plt.show()
    break

## Models

In [5]:
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=4,  
    channels=(32, 64, 128, 256, 512),
    strides=(2, 2, 2, 2),
    num_res_units=4,
    norm="instance",  # Use Instance Normalization
    dropout=0.2       # Add dropout
)
# model = UNETR(in_channels=1, out_channels=4, img_size=(256,128,258), feature_size=32, norm_name='batch')
model = model.to(DEVICE)

## Loss & Optimizer

In [6]:
# class_weights = train_dataset.calculate_class_weights_log(num_classes=4).to(DEVICE)
# criterion = DiceCrossEntropyLoss(dice_weight=1.0, ce_weight=0.0, is_3d=True, class_weights=class_weights)
criterion = DiceFocalLoss(alpha=[0.05, 0.5, 0.3, 0.3], gamma=2, is_3d=True, ignore_background=False)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-7)

## Training

In [7]:
best_avg_dice = 0
for epoch in range(EPOCHS):
    train_avg_loss, train_avg_dice, train_csf_dice, train_gm_dice, train_wm_dice = train_3d(model, train_loader, criterion, optimizer, DEVICE, epoch, EPOCHS, NUM_CLASSES)
    val_avg_loss, val_avg_dice, val_csf_dice, val_gm_dice, val_wm_dice = validate_3d(model, val_loader, criterion, DEVICE, epoch, EPOCHS, NUM_CLASSES)
    scheduler.step()
    if val_avg_dice > best_avg_dice:
        best_avg_dice = val_avg_dice
        torch.save(model.state_dict(), 'best_model_3d.pth')
        print(f'Best model saved with dice score: {best_avg_dice}\n')

Epoch 195, Loss: 0.0459
Epoch 195, Dice: 0.9229 CSF Dice: 0.8781 GM Dice: 0.9133 WM Dice: 0.9032


Validation Epoch 195/300:   0%|          | 0/5 [00:00<?, ?it/s]

Validation Epoch 195/300: 100%|██████████| 5/5 [00:01<00:00,  3.01it/s, Loss=0.0205, Avg Dice=0.936, CSF Dice=0.919, GM Dice=0.928, WM Dice=0.9]  


Epoch 195, Loss: 0.0409
Epoch 195, Dice: 0.9323 CSF Dice: 0.8918 GM Dice: 0.9256 WM Dice: 0.9149



Training Epoch 196/300: 100%|██████████| 10/10 [00:04<00:00,  2.38it/s, Loss=0.0265, Avg Dice=0.909, CSF Dice=0.813, GM Dice=0.926, WM Dice=0.898]


...

Epoch 299, Loss: 0.0380
Epoch 299, Dice: 0.9372 CSF Dice: 0.8997 GM Dice: 0.9304 WM Dice: 0.9218



Training Epoch 300/300: 100%|██████████| 10/10 [00:04<00:00,  2.33it/s, Loss=0.0207, Avg Dice=0.929, CSF Dice=0.883, GM Dice=0.917, WM Dice=0.921]


Epoch 300, Loss: 0.0424
Epoch 300, Dice: 0.9285 CSF Dice: 0.8851 GM Dice: 0.9199 WM Dice: 0.9123


Validation Epoch 300/300: 100%|██████████| 5/5 [00:01<00:00,  2.72it/s, Loss=0.0188, Avg Dice=0.941, CSF Dice=0.927, GM Dice=0.934, WM Dice=0.907]

Epoch 300, Loss: 0.0380
Epoch 300, Dice: 0.9372 CSF Dice: 0.8998 GM Dice: 0.9304 WM Dice: 0.9218

